# Visualization of skeleton crops with Colorado

In [8]:
import sys, os
import numpy as np
import matplotlib.pyplot as plt

print(sys.path)

# Import the aims module
from soma import aims
# the brainplot package
import colorado as cld

print(sys.version)

['/volatile/jc225751/Runs/14_regularize_training/Program/dicoFolding/notebooks', '', '/casa/host/src/development/casa-distro/master/python', '/casa/host/src/populse/populse_db/master/python', '/casa/host/src/capsul/master', '/casa/host/build/python', '/casa/home/.vscode/extensions/ms-toolsai.jupyter-2021.6.832593372/pythonFiles', '/casa/home/.vscode/extensions/ms-toolsai.jupyter-2021.6.832593372/pythonFiles/lib/python', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/home/jc225751/Runs/14_regularize_training/Program/dicoFolding/venv/lib/python3.6/site-packages', '/volatile/jc225751/Runs/14_regularize_training/Program/dicoFolding', '/casa/home/.local/lib/python3.6/site-packages', '/home/jc225751/Program/deep_folding', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.6/dist-packages/IPython/extensions', '/casa/home/.ipython']
3.6.9 (default, Jan 26 2021, 15:33:00) 
[GCC 8.4.0]


In [27]:
import hydra

In [31]:
@hydra.main(config_name='config', config_path="config")
def postprocessing_results(config):
    config = process_config(config)

    # Sets seed for pseudo-random number generators
    # in: pytorch, numpy, python.random
    # seed_everything(config.seed)

    data_module = DataModule(config)
    data_module.setup(stage='validate')

    # Show the views of the first batch
    # fig = plt.figure(figsize=(4., 8.), dpi=400)
    # grid = ImageGrid(fig, 111,
    #                 nrows_ncols = (config.batch_size//4, 8),
    #                 axes_pad=0.2,)
    # (inputs, filenames) = next(iter(data_module.val_dataloader()))
    # input_i = inputs[:, 0, :]
    # input_j = inputs[:, 1, :]
    # print("input_i : {}".format(np.unique(input_i)))
    # print("input_j : {}".format(np.unique(input_j)))
    # images = []
    # np.save("input_i.npy", input_i[0, 0, :, :, :])
    # np.save("input_j.npy", input_j[0, 0, :, :, :])
    # vol_i = aims.Volume(1, 80, 80, 80, dtype=np.int32)
    # np.asarray(vol_i)[:] = input_i[0, :, :, :, :]
    # aims.write(vol_i, 'input_i.nii')
    # print(np.unique(input_i[0, :, :, :, :]))
    # for i in range(config.batch_size):
    #     images.append(input_i[i, 0, input_i.shape[2]//2, :, :])
    #     images.append(input_j[i, 0, input_i.shape[2]//2, :, :])
    # for ax, im in zip(grid, images):
    #     ax.imshow(im)
    #     ax.axis('off')
    #     ax.set_title(np.unique(im)[1], fontsize=4)
    # plt.show()
    
    # Show the views of the first skeleton after each epoch
    model = ContrastiveLearner(config,
                            mode="encoder",
                            sample_data=data_module)
    model = model.load_from_checkpoint(config.checkpoint_path,
                                       config=config,
                                       mode="encoder",
                                       sample_data=data_module)
    summary(model, tuple(config.input_size), device="cpu")
    trainer = pl.Trainer(
        gpus=1,
        max_epochs=config.max_epochs,
        logger=tb_logger,
        flush_logs_every_n_steps=config.nb_steps_per_flush_logs,
        resume_from_checkpoint=config.checkpoint_path)
    dataset_val = data_module.dataset_val
    trainer.validate(model, data_module.val_dataloader()) 
    embeddings, filenames = model.compute_representations(data_module.val_dataloader())
    
    return embeddings, data_module



In [33]:
embeddings, data_module = postprocessing_results()

usage: ipykernel_launcher.py [--help] [--hydra-help] [--version]
                             [--cfg {job,hydra,all}] [--package PACKAGE]
                             [--run] [--multirun] [--shell-completion]
                             [--config-path CONFIG_PATH]
                             [--config-name CONFIG_NAME]
                             [--config-dir CONFIG_DIR] [--info]
                             [overrides [overrides ...]]
ipykernel_launcher.py: error: argument --shell-completion/-sc: ignored explicit argument '9002'


SystemExit: 2

### Transformation of .npy saved skeletons to .bck

#### Test with crop obtained with linear interpolation

In [24]:
crop = aims.read('/neurospin/dico/lguillon/skeleton/sts_crop/right_hemi/normalized_crop/Rskeleton_107422_normalized_crop.nii.gz')
cnt=0
im2 = np.copy(crop)
for k in range(len(im2)):
    for i in range(len(im2[k])):
        for j in range(len(im2[k][i])):
            vox = im2[k][i][j]
            if vox>1: # On est sur un sillon
                if k>0 and j>0 and i>0 and k<43 and i<71 and j<68:
                    if im2[k-1][i][j]==11 or im2[k+1][i][j]==11 or im2[k][i-1][j]==11 or im2[k][i+1][j]==11 or im2[k][i][j-1]==11 or im2[k][i][j+1]==11:
                        cnt+=1
                        im2[k][i][j]=11
                        

In [25]:
im2[im2==0] = 10
im2[im2!=10] =0
input_vol = aims.Volume(im2)
cld.draw_as_mesh(input_vol)

Here sulci are very well represented, and even a bit too bold.

It would be interesting to try with Bastien's resampling or resampling with priority order.